# Importing libraries

In [1]:
import sys
import os
import json
import numpy as np
import torch
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils_data import normalize_new_row_with_mean_std
import asyncio
import threading
from websocket_client import fetch_market_data, get_latest_ohlc_volume, market_data_store, get_market_summary
import time
from utils_data import TechnicalIndicatorWindow
from utils_data import  UpstoxTrader, TradeMonitoringBot
import pytz
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

# Getting Real Time Data

In [2]:
# Run the WebSocket client in a background thread
async def start_connection():
    while True:
        try:
            await fetch_market_data(symbol="SUZLON")
        except Exception as e:
            print(f"Error: {e}, reconnecting in 5 seconds...")
            await asyncio.sleep(5)

def start_ws_loop():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_connection())

# Start thread
thread = threading.Thread(target=start_ws_loop)
thread.start()
# Give it time to connect and start receiving
time.sleep(5)


{'status': 'success', 'data': {'authorizedRedirectUri': 'wss://wsfeeder-api.upstox.com/market-data-feeder/v3/upstox-developer-api/feeds?requestId=02fea7f0-01a0-4b77-9d4b-57902047fc62&code=c2lBd-bbc2d351-d6ef-456a-bbbf-57f89c39a297', 'authorized_redirect_uri': 'wss://wsfeeder-api.upstox.com/market-data-feeder/v3/upstox-developer-api/feeds?requestId=02fea7f0-01a0-4b77-9d4b-57902047fc62&code=c2lBd-bbc2d351-d6ef-456a-bbbf-57f89c39a297'}}
Connection established


Error: no close frame received or sent, reconnecting in 5 seconds...
{'status': 'success', 'data': {'authorizedRedirectUri': 'wss://wsfeeder-api.upstox.com/market-data-feeder/v3/upstox-developer-api/feeds?requestId=f70965f4-ea48-48c6-9542-5cae4235cebd&code=ud4WO-f892b867-1ff7-49d8-bf93-2d269d6621f1', 'authorized_redirect_uri': 'wss://wsfeeder-api.upstox.com/market-data-feeder/v3/upstox-developer-api/feeds?requestId=f70965f4-ea48-48c6-9542-5cae4235cebd&code=ud4WO-f892b867-1ff7-49d8-bf93-2d269d6621f1'}}
Connection established
Error: no close frame received or sent, reconnecting in 5 seconds...
Error: HTTPSConnectionPool(host='api.upstox.com', port=443): Max retries exceeded with url: /v3/feed/market-data-feed/authorize (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001EF95AF4980>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')), reconnecting in 5 seconds...
{'status': 'success', 'data': {'authorizedRedirectUri': 'wss://wsfeeder

In [10]:
latest = get_latest_ohlc_volume("SUZLON")
latest

{'timestamp': datetime.datetime(2025, 6, 19, 15, 29),
 'open': 62.48,
 'high': 62.48,
 'low': 62.25,
 'close': 62.42,
 'volume': '975610'}

In [4]:
# Import Norm Param for converting real time data to normalized data
with open("C:/Users/srija/Assignment/Trading/json_files/suzlon_14_june_norm_params.json", "r") as f:
    norm_params = json.load(f)


In [5]:
def get_state(row):
    state = np.array([
        row['time'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['MA50'],
        row['RSI'],
        row['MACD'],
        row['BB_upper'],
        row['BB_lower'],
        row['ADX'],
        row['CCI'],
        row['ATR'],
        row['ROC'],
        row['OBV']
    ], dtype=np.float32)
    return state

# Loading Models

In [ ]:
from Models.DQN import DQN,DQNAgent
from trading_environment import StockTradingEnv

policy_net = DQN(16, 3)
target_net = DQN(16, 3)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

device = torch.device('cpu') #'cuda' if torch.cuda.is_available() else 

policy_net.to(device)
target_net.to(device)

model_path = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june\suzlon_14_june_1161.pth"
policy_net.load_state_dict(torch.load(model_path))
policy_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

# Action time

In [7]:
trader = UpstoxTrader()
symbol ="SUZLON"
ti_window = TechnicalIndicatorWindow(window_size=28)

In [ ]:
bot = TradeMonitoringBot(trader)
india = pytz.timezone('Asia/Kolkata')
now_ist = datetime.now(india)
market_close_time = now_ist.replace(hour=15, minute=9, second=0, microsecond=0)
csv_path = Path(f'C:/Users/srija/Assignment/Trading/json_files/suzlon_{datetime.now(india).date()}.csv')

while True:
    t1 = time.time()
    now_ist = datetime.now(india)
    try:
        market_status = trader.MarketStatus()
    except Exception as e:
        print(f"Error fetching market status: {e}")
        
    if market_status is False or now_ist >= market_close_time:
            trader.exitAllPositions()
            trader.CancelOrder(order_id=order_id['stop_loss_order'])
            trader.CancelOrder(order_id=order_id['target_order'])
            bot.stop_order_monitoring()
            print("Market is closed.")
            break
    new_bar = get_latest_ohlc_volume(symbol=symbol)

    file_exists = csv_path.exists()  # Save the new bar data to CSV
    with open(csv_path, 'a') as f:
        if not file_exists:
            f.write("time,open,high,low,close,volume\n")
        f.write(f"{new_bar['timestamp']},{new_bar['open']},{new_bar['high']},{new_bar['low']},{new_bar['close']},{new_bar['volume']}\n")

    ti_window.update(new_bar)
    features = ti_window.get_feature_row()
    if features is None:
        print("No features available yet.")
        time.sleep(60)
        continue
    if not bot.has_active_orders():
        features = features.rename({'timestamp': 'time'})
        normalized_row = normalize_new_row_with_mean_std(features, norm_params)
        state = get_state(normalized_row)
        t2 = time.time()
        print(f"Time taken to get state: {t2 - t1:.2f} seconds")
        
        with torch.no_grad():
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            q_values = policy_net(state)
            action = q_values.argmax().item()
        
        t3 = time.time()
        print(f"Time taken for inference: {t3 - t2:.2f} seconds")
        
        if action != 0:  # If action is BUY or SELL
            order_id = {}
            with ThreadPoolExecutor(max_workers=2) as executor:
                future_available_cash = executor.submit(trader.get_margin)
                future_latest_price = executor.submit(get_market_summary, symbol)
                available_cash = future_available_cash.result()
                latest_price = future_latest_price.result()['latest_price']
            
            quantity = int(available_cash * 0.9 * 2.5 / latest_price)
            if quantity <= 10:
                print("Insufficient funds to place an order.")
            else:
                t4 = time.time()
                print(f"Time taken to prepare order: {t4 - t3:.2f} seconds")
                
                try:
                    if action == 1:  # BUY
                        print("Placing buy order.")
                        with ThreadPoolExecutor(max_workers=3) as executor:
                            future_market = executor.submit(trader.IntraMarketOrder, symbol, quantity, "BUY")
                            future_target = executor.submit(trader.IntraLimitOrder, symbol, quantity, "SELL", latest_price * 1.004)
                            future_stop_loss = executor.submit(trader.IntraDayStopLossOrder, symbol, quantity, "SELL", latest_price * 0.99)

                            market_order = future_market.result()
                            target_order = future_target.result()
                            stop_loss_order = future_stop_loss.result()

                    elif action == 2:  # SELL
                        print("Placing sell order.")
                        with ThreadPoolExecutor(max_workers=3) as executor:
                            future_market = executor.submit(trader.IntraMarketOrder, symbol, quantity, "SELL")
                            future_target = executor.submit(trader.IntraLimitOrder, symbol, quantity, "BUY", latest_price * 0.996)
                            future_stop_loss = executor.submit(trader.IntraDayStopLossOrder, symbol, quantity, "BUY", latest_price * 1.01)

                            market_order = future_market.result()
                            target_order = future_target.result()
                            stop_loss_order = future_stop_loss.result()
                    
                    order_id['market_order'] = market_order['data']['order_id']
                    order_id['target_order'] = target_order['data']['order_id']
                    order_id['stop_loss_order'] = stop_loss_order['data']['order_id']
                    
                    t5 = time.time()
                    print(f"Time taken to place orders: {t5 - t4:.2f} seconds")
                    
                    # Start monitoring orders in background thread
                    bot.start_order_monitoring(order_id)
                    
                except Exception as e:
                    print(f"An error occurred while placing orders: {e}")
    else:
      print("Orders are being monitored in background, continuing data updates...")
      
    

    # Wait for the next minute to start the next iteration
    next_minute = (datetime.now(india) + timedelta(minutes=1)).replace(second=0, microsecond=0)
    while datetime.now(india) < next_minute:
        time.sleep(0.5)
    time.sleep(4)


No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
Time taken to get state: 0.43 seconds
Time taken for inference: 0.09 seconds
Time taken to prepare order: 0.46 seconds
Placing sell order.
Error: HTTPSConnectionPool(host='api-hft.upstox.com', port=443): Max retries exceeded with url: /v2/order/place (Caused by SSLError(